In [141]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [ ]:
# 函数：对每个onehot_sequence字符串进行重新排序
def reorder_onehot_sequence(sequence):
    # 将字符串转化为列表，按4个数字一组进行切割
    chunks = [sequence[i:i+4] for i in range(0, len(sequence), 4)]
    
    # 创建新的四个部分，分别对应每组的第一、第二、第三、第四个数字
    part1 = ''.join([chunk[0] for chunk in chunks])  # 每组的第一个数字
    part2 = ''.join([chunk[1] for chunk in chunks])  # 每组的第二个数字
    part3 = ''.join([chunk[2] for chunk in chunks])  # 每组的第三个数字
    part4 = ''.join([chunk[3] for chunk in chunks])  # 每组的第四个数字
    
    # 合并四部分并返回新的字符串
    return part1 + part2 + part3 + part4

In [ ]:
# 函数：将DNA序列转化为one-hot编码的展平向量
def sequence_to_onehot(sequence):
    # 使用ATCG四位one hot
    base_to_onehot = {
        'A': [1, 0, 0, 0],  # A -> [1, 0, 0, 0]
        'a': [1, 0, 0, 0],
        'T': [0, 1, 0, 0],  # T -> [0, 1, 0, 0]
        't': [0, 1, 0, 0], 
        'C': [0, 0, 1, 0],  # C -> [0, 0, 1, 0]
        'c': [0, 0, 1, 0],
        'G': [0, 0, 0, 1],   # G -> [0, 0, 0, 1]
        'g': [0, 0, 0, 1]
    }
    onehot = []
    for base in sequence:
        onehot.extend(base_to_onehot.get(base, [0, 0, 0, 0]))  # 获取one-hot编码
    return onehot

In [ ]:
def sample_preprocessing(sample_name, ColumnAttention = False):
    NomalSamples = 'NomalSamples'
    if (ColumnAttention):
        NomalSamples = 'SelfAttentionSamples'
    Cov_Mod_path = f'/BioII/lulab_b/huangkeyun/zhangys/alkb-seq/resources/medium_data/{sample_name}_recenter_Cov_Mod.txt'
    sequence_df_path = f'/BioII/lulab_b/huangkeyun/zhangys/alkb-seq/resources/medium_data/{sample_name}_recenter.fa.csv'
    # sample_name = os.path.basename(Cov_Mod_path).split('_')[0]

    # 读取正在处理样本的两个文件，规范sampl列
    Cov_Mod_df = pd.read_csv(Cov_Mod_path,sep = '\t')
    sequence_df = pd.read_csv(sequence_df_path, names = ['sample', 'score1', 'score2', 'struc', 'sequence'])
    sequence_df['sample'] = Cov_Mod_df['sample'].str.split('>').str[0]

    # 从sequence文件删除存在N额行，并根据sample列从Cov_Mod文件中同步删除行
    # 记录包含'N'的行的 sample 信息
    original_row_count = len(sequence_df)
    deleted_samples = sequence_df[sequence_df['sequence'].str.contains('N|n')]['sample'].tolist()
    sequence_df = sequence_df[~sequence_df['sequence'].str.contains('N|n')]
    # 从 Cov_Mod_df 中删除相应的行
    Cov_Mod_df = Cov_Mod_df[~Cov_Mod_df['sample'].isin(deleted_samples)]
    # 计算行数变化
    new_row_count = len(sequence_df)
    deleted_rows = original_row_count - new_row_count
    print(f"删除了 {deleted_rows} 行，其中 sample 列的样本为：{deleted_samples}")

    # 根据sample列生成label，保存为csv文件
    Cov_Mod_df['last_part'] = Cov_Mod_df['sample'].str.split('|').str[-1]
    Cov_Mod_df['label'] = Cov_Mod_df['last_part'].map({'same': 0, 'diff': 1})
    labels_df = Cov_Mod_df[["sample", "label"]]
    labels_output_path = f'/BioII/lulab_b/huangkeyun/zhangys/alkb-seq/resources/{NomalSamples}/labels/{sample_name}_labels.csv'
    labels_df.to_csv(labels_output_path,index = None)

    # 创建用于最终输出的sample_prepared_df，向其注入Cov_Mod文件列和sequence产生的onehot编码
    sample_prepared_df = Cov_Mod_df.iloc[:, 0:-1-1]


    # 将每个DNA序列转换为one-hot编码并展平
    sequence_df['onehot_sequence'] = sequence_df['sequence'].apply(sequence_to_onehot)
    sample_prepared_df = pd.concat([sample_prepared_df, sequence_df['onehot_sequence']], axis=1)
    # 去除分隔符号
    sample_prepared_df['onehot_sequence'] = sample_prepared_df['onehot_sequence'].apply(lambda x: ''.join(map(str, x)))

    # optional： 如果要生成列注意力样本，对每一行的 'onehot_sequence' 列进行重新排序
    if (ColumnAttention == True):
        sample_prepared_df['onehot_sequence'] = sample_prepared_df['onehot_sequence'].apply(reorder_onehot_sequence)

    # 将字符串分隔为列
    sample_prepared_df[['seq' + str(i+1) for i in range(160)]] = sample_prepared_df['onehot_sequence'].apply(lambda x: pd.Series(list(x)))
    sample_prepared_df = sample_prepared_df.drop(columns=['onehot_sequence'])
    sample_prepared_output_path = f'/BioII/lulab_b/huangkeyun/zhangys/alkb-seq/resources/{NomalSamples}/samples/{sample_name}_sample_prepared.csv'
    sample_prepared_df.to_csv(sample_prepared_output_path, sep = ',', index = None)

In [ ]:
sample_list = ["SRR11004103","SRR11004104","SRR11004105","SRR11004118","SRR11004119","SRR11004120","SRR11004130","SRR11004131"]
for sample_name in sample_list:
    sample_preprocessing(sample_name, ColumnAttention = False)

In [ ]:
sample_list = ["SRR11004103","SRR11004104","SRR11004105","SRR11004118","SRR11004119","SRR11004120","SRR11004130","SRR11004131"]
for sample_name in sample_list:
    sample_preprocessing(sample_name, ColumnAttention = True)